<a href="https://colab.research.google.com/github/eldananyss21/assignment-3-classification/blob/main/code/assignment3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: Text Classification

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset
import torch
from sentence_transformers import SentenceTransformer
from sklearn import svm

print("Using GPU:", torch.cuda.is_available())

# IMPORT DATA
train = pd.read_csv("./data/train.csv")
dev = pd.read_csv("./data/dev.csv")
test = pd.read_csv("./data/test.csv")

# ENCODE LABELS
label_encoder = LabelEncoder()
label_encoder.fit(train["label"].tolist() + dev["label"].tolist())
label_names = label_encoder.classes_.tolist()
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for i, name in enumerate(label_names)}

# TOKENIZE + CREATE DATASETS FOR TRACK 2 & TRACK 3
def tokenize_dataframes(model_name, train, dev, test, train_dev):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_fn(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    train_ds = Dataset.from_pandas(train).map(tokenize_fn, batched=True).rename_column("label", "labels")
    dev_ds = Dataset.from_pandas(dev).map(tokenize_fn, batched=True).rename_column("label", "labels")
    test_ds = Dataset.from_pandas(test).map(tokenize_fn, batched=True)
    train_dev_ds = Dataset.from_pandas(train_dev).map(tokenize_fn, batched=True).rename_column("label", "labels")

    return tokenizer, train_ds, dev_ds, test_ds, train_dev_ds